In [82]:
import numpy as np
import pandas as pd
import math

In [83]:
def monthly_return(return_yearly:float, method:str = "geometric"):
  """
  Calculate the monthly return of an expected yearly return.
  默认使用几何平均数计算每月收益率。
  可选算法：geometric, arithmetic(算术平均数)
  """
  if method == "geometric":
    return pow(1 + return_yearly, 1 / 12) - 1
  elif method == "arithmetic":
    return return_yearly / 12
  else:
    raise ValueError("method should be either 'geometric' or 'arithmetic'")
  

In [84]:
def automatic_investment(return_monthly: float, investment_monthly: float, horizon: float, initial_balance: float = 0, increment: float = 0, incre_period: int = 0):
  """
  Calculate the balance after a certain period of time with a certain monthly investment and monthly return.
  suppose the investment is made at the start of each month.
  
  initial_balance: default 0.
  
  是否增加定投额的判断
  1.是否经过了一年：(i+1) % 12 == 0
  2.增加定投额是否为0：increment != 0
  3.当前年限是否在增加定投额年限内：year_num <= incre_period
  4.判断是否为第一年，第一年不进行增加定投额的操作，增加定投额从第二年开始：year_num != 0
  """
  month_num = horizon * 12
  balance = initial_balance + investment_monthly
  

  for i in range(month_num):
    year_num = i // 12
    if (i+1) % 12 == 0 and increment != 0 and year_num <= incre_period and year_num != 0:  #判断是否增加定投额
      investment_monthly += increment
        
    balance = balance * (1 + return_monthly) + investment_monthly

  return balance

In [85]:
def back_to_present(target:str, target_value: float, return_monthly: float, horizon: int, investment_monthly: float = 0, initial_balance: float = 0):
  """
  Calculate the monthly investment value or expected monthly return.
  suppose the investment is made at the start of each month.
  
  target: "num" or "rate"
  “num”：目标为金额
  "rate": 目标为总收益率
  "horizon": 投资年限，单位年，整数
  "investment_monthly": 每月定投金额，单位（万）
  "return_monthly": 每月收益率
  "initial_balance": 初始本金，单位（万）
  
  """
  # 要求return_monthly or investment monthly
  month_num = horizon * 12
  
  if target == "num":
    monthly_num = (target_value - initial_balance * pow(1 + return_monthly, month_num)) * \
                  return_monthly / (pow(1 + return_monthly, month_num - 1) - 1)
    return monthly_num
    
  elif target == "rate":
    pass
  
  else:
    raise ValueError("target should be either 'num' or 'rate'")
    
  

In [86]:
def time_to_target(target_value: float, return_monthly: float, investment_monthly: float, init: float = 0):
  """
  Calculate the time needed to reach the target value with a certain monthly investment and monthly return.
  suppose the investment is made at the start of each month.
  
  init: default 0.
  
  """
  month_num = 0
  balance = init

  while balance < target_value:
    balance = balance * (1 + return_monthly) + investment_monthly
    month_num += 1

  return month_num/12

In [87]:
yearly_return = 0.10 # 期待的年化收益率
investment_horizon = 20 # 投资年限
monthly_investment = 10 # 每月定投金额，单位（万）
initial_balance = 0 # 初始本金
increment = 0 # 每年增加的定投金额，单位（万）
increment_period = 0 # 增加定投金额的年限，比如预计可以连续5年增加，则填5

In [88]:
print(f"经过{investment_horizon}年的总余额是： {automatic_investment(monthly_return(yearly_return), monthly_investment, investment_horizon, initial_balance, increment, increment_period):.2f} \n \
共盈利 {automatic_investment(monthly_return(yearly_return), monthly_investment, investment_horizon, initial_balance, increment, increment_period) - (investment_horizon*12*monthly_investment+12*increment_period*(2*increment + (increment_period-1)*increment)*(1/2)):.2f}") # 投资总额的变化需要考虑到定投金额是一个等差数列

经过20年的总余额是： 7249.87 
 共盈利 4849.87


In [89]:
print(f"每月需要投入 {back_to_present('num', 10000, monthly_return(yearly_return), investment_horizon, monthly_investment, initial_balance):.2f}")

每月需要投入 14.05
